## VGG-11 Initial Training with CIFAR-100 

In [1]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.optim.lr_scheduler import LambdaLR
import random
import numpy as np
import os

In [2]:
def fix_random_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Fixed random seed: {seed}")

fix_random_seed(42)

# For deterministic DataLoader behavior
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

Fixed random seed: 42


In [3]:
vgg11 = models.vgg11(weights=models.VGG11_Weights.IMAGENET1K_V1)
vgg11.classifier[6] = torch.nn.Linear(4096, 100)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"
vgg11 = vgg11.to(device)

base_lr = 0.04
weight_decay = 5e-4
warmup_epochs = 3
num_epochs = 9
optim = torch.optim.SGD(
        vgg11.parameters(),
        lr=base_lr,
        momentum=0.9,
        weight_decay=weight_decay
    )

def lr_lambda(current_epoch):
    if current_epoch < warmup_epochs:
        # Linear warmup from 0 -> 1
        return float(current_epoch + 1) / float(max(1, warmup_epochs))
    else:
        # Cosine annealing from 1 -> 0
        progress = (current_epoch - warmup_epochs) / float(max(1, num_epochs - warmup_epochs))
        return 0.5 * (1.0 + torch.cos(torch.tensor(progress * 3.1415926535)))

scheduler = LambdaLR(optim, lr_lambda=lr_lambda)

criterion = torch.nn.CrossEntropyLoss()

In [4]:
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]
BATCH_SIZE = 256
NUM_WORKERS = 0
VAL_RATIO = 0.05

g = torch.Generator()
g.manual_seed(42)

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        imagenet_mean, imagenet_std
    ),
])

def split_train_val(dataset, ratio=VAL_RATIO, generator=g):
    total_size = len(dataset)
    val_size = int(ratio * total_size)
    train_size = total_size - val_size
    return random_split(dataset, [train_size, val_size], generator=generator)

train_dataset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform
)

train_dataset, val_dataset = split_train_val(train_dataset, VAL_RATIO, g)

test_dataset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform
)

print("Training set size:", len(train_dataset))
print("Validation set size:", len(val_dataset))
print("Test set size:", len(test_dataset))

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    torch.manual_seed(worker_seed)
    torch.cuda.manual_seed_all(worker_seed)

def get_loader(dataset, shuffle):
    return DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        worker_init_fn=seed_worker,
        generator=g
    )

train_loader = get_loader(train_dataset, shuffle=True)
val_loader = get_loader(val_dataset, shuffle=False)
test_loader = get_loader(test_dataset, shuffle=False)

Training set size: 47500
Validation set size: 2500
Test set size: 10000


In [5]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.to(device)
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / total
    accuracy = 100. * correct / total
    return avg_loss, accuracy

In [6]:
def evaluate(model, test_loader, device):
    model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            preds = model(images)
            preds = torch.argmax(preds, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total * 100
    return accuracy

In [7]:
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(vgg11, train_loader, criterion, optim, device)
    scheduler.step()
    val_acc = evaluate(vgg11, val_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}")

KeyboardInterrupt: 

In [ ]:
acc = evaluate(vgg11, test_loader, device)
print(f"Accuracy: {acc:.2f}%")

Accuracy: 0.88%
